In [62]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
import yfinance as yf
import numpy as np
import pyfolio as pf
from io import StringIO
import sklearn as skn
from scipy import stats
import pandas_datareader.data as pdr
import riskfolio as rp
import cvxpy as cp


In [7]:
def loadStooqData(ticker: str,start, frequency='d'):
    url = f'https://stooq.pl/q/d/l/?s={ticker}&i=d'
    response = requests.get(url)  
    data = pd.read_csv(StringIO(response.text))
    data.set_index('Data', inplace=True)
    data = data[start:]
    return data['Zamkniecie']

def loadYahooData(ticker, start, frequency = '1d'):      
    ticker  = yf.Ticker(ticker)
    hist = ticker.history(start=start,interval=frequency)
    price = hist['Close']       
    return price

def sharp(returns):
    return returns.mean()/returns.std()

def assets_performance(returns: pd.DataFrame):
    return returns.agg(['mean', 'std', 'median', 'skew', 'kurtosis', sharp])

def beta(X, Y):
       
    X_cov = X.cov()

    XY = np.append(X,np.expand_dims(Y,axis=1),axis=0)
    XY_cov = np.cov(XY)

    #XY_sub = XY_cov.iloc[-1,0:2]

    #inv = np.linalg.inv(X_cov)
    #B = inv @ XY_sub
    return  XY_cov

In [135]:
yahoo_tickers = ['ISAC.L', 'CORP.L']

stooq_tickers = ['XAUPLN', '^TBSP', 'MWIG40TR', 'USDPLN', 'EURPLN', 'PLOPLN3M']

#na przyszłość EAFA,EEM,ACWI



In [136]:
base_prices = pd.DataFrame({})
start = '2012-01-01'

for ticker in stooq_tickers:
    base_prices[ticker] = loadStooqData(ticker,start)

for ticker in yahoo_tickers:
    base_prices[ticker] = loadYahooData(ticker,start)

base_prices.index = pd.DatetimeIndex(base_prices.index)


In [137]:
#get monthly data
monthly_base_prices = base_prices.resample('m').last()

In [138]:
monthly_base_returns = np.log(monthly_base_prices/monthly_base_prices.shift(1))
assets_performance(monthly_base_returns)

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ISAC.L,CORP.L
mean,0.002463,0.002636,0.008253,0.001512,0.000321,0.002168,0.007260,-0.001095
std,0.043043,0.013529,0.050501,0.031244,0.016053,0.171776,0.039464,0.019773
median,-0.000669,0.003073,0.012215,0.002273,-0.000887,0.000000,0.012231,0.001980
skew,0.000982,0.292052,-0.690601,0.219275,0.469020,2.587719,-0.679594,-0.711760
kurtosis,0.450405,5.420861,3.290414,0.405569,0.455419,29.249657,1.021297,2.587958
sharp,0.057230,0.194831,0.163430,0.048384,0.020004,0.012619,0.183955,-0.055390


In [139]:
#calculate pln returns
pln_prices = monthly_base_prices
pln_prices['ISAC.L'] = monthly_base_prices['ISAC.L'] * monthly_base_prices['USDPLN']
pln_prices['CORP.L'] = monthly_base_prices['CORP.L'] * monthly_base_prices['USDPLN']
pln_returns = np.log(pln_prices/pln_prices.shift(1))
assets_performance(pln_returns)

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ISAC.L,CORP.L
mean,0.002463,0.002636,0.008253,0.001512,0.000321,0.002168,0.008771,0.000616
std,0.043043,0.013529,0.050501,0.031244,0.016053,0.171776,0.032784,0.023803
median,-0.000669,0.003073,0.012215,0.002273,-0.000887,0.000000,0.007464,0.000084
skew,0.000982,0.292052,-0.690601,0.219275,0.469020,2.587719,-0.031383,0.242463
kurtosis,0.450405,5.420861,3.290414,0.405569,0.455419,29.249657,0.623673,-0.385246
sharp,0.057230,0.194831,0.163430,0.048384,0.020004,0.012619,0.267549,0.025867


In [140]:
pln_returns.corr()

,XAUPLN,^TBSP,MWIG40TR,USDPLN,EURPLN,PLOPLN3M,ISAC.L,CORP.L
XAUPLN,1.000000,0.057969,-0.188596,0.395596,0.401594,0.028242,0.042953,0.487553
^TBSP,0.057969,1.000000,0.088809,-0.234736,-0.152227,-0.410785,0.023677,0.162884
MWIG40TR,-0.188596,0.088809,1.000000,-0.469890,-0.501510,-0.055884,0.431060,-0.107345
USDPLN,0.395596,-0.234736,-0.469890,1.000000,0.756308,0.078503,0.240910,0.743578
EURPLN,0.401594,-0.152227,-0.501510,0.756308,1.000000,0.025808,0.063840,0.554084
PLOPLN3M,0.028242,-0.410785,-0.055884,0.078503,0.025808,1.000000,0.009951,-0.000751
ISAC.L,0.042953,0.023677,0.431060,0.240910,0.063840,0.009951,1.000000,0.484318
CORP.L,0.487553,0.162884,-0.107345,0.743578,0.554084,-0.000751,0.484318,1.000000


In [147]:
pln_returns.mean() * 12

XAUPLN      0.029560
^TBSP       0.031630
MWIG40TR    0.099040
USDPLN      0.018140
EURPLN      0.003854
PLOPLN3M    0.026012
ISAC.L      0.105256
CORP.L      0.007388
dtype: float64

In [152]:
expected_returns = pd.Series({
'ISAC.L': 0.085/12,
'^TBSP': 0.035/12,
'XAUPLN': 0.054/12,
'MWIG40TR': 0.08/12,
'CORP.L': 0.037/12,
'PLOPLN3M': 0.03/12
})





In [182]:
assets = ['ISAC.L', '^TBSP', 'XAUPLN', 'MWIG40TR', 'CORP.L', 'PLOPLN3M']
#
cov_matrix = pln_returns[assets].cov()

* load data
* describe data in local currency
* transform data do pln
* describe data in pln

In [290]:
portfolio = rp.Portfolio(pd.DataFrame(pln_returns[assets]))
#pln_returns[assets]

portfolio.mu = expected_returns
portfolio.cov = cov_matrix

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = False # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = portfolio.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
weights,0.051164,0.773241,0.029394,0.029713,0.08716,0.029328


In [291]:
w = portfolio.optimization(model=model, rm=rm, obj='MaxRet', rf=rf, l=l, hist=hist)

display(np.round(w.T,2))

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M
weights,1.0,0.0,0.0,0.0,0.0,0.0


In [292]:
def frontier_performance(frontier,mu,cov):
    result = frontier.copy()
    result['Returns'] = np.round(frontier.to_numpy() @ mu *12,2)
    result['Std. deviations'] = frontier.apply(lambda x: np.round(np.sqrt(x.to_numpy().T @ cov @ x.to_numpy())* np.sqrt(12),2),axis=1)
    return result


In [311]:
def shrinkedCovariance(returns: pd.DataFrame, w: int):
    std = np.diag(returns.std())
    corr = returns.corr('pearson')
    shrinked_corr = (1-w)*corr + np.ones_like(corr)*w
    result = std @ shrinked_corr @ std
    return result

shrin_param = 0.75

In [312]:
points = 10
frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10 = np.round(frontier.T,2)

In [313]:
frontier_performance(frontier_10, expected_returns,cov_matrix)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations
0,0.00,1.00,0.0,0.0,0.0,0.0,0.04,0.05
1,0.21,0.79,0.0,0.0,0.0,0.0,0.05,0.04
2,0.33,0.67,0.0,0.0,0.0,0.0,0.05,0.05
3,0.43,0.57,0.0,0.0,0.0,0.0,0.06,0.06
4,0.53,0.47,0.0,0.0,0.0,0.0,0.06,0.06
5,0.63,0.37,0.0,0.0,0.0,0.0,0.07,0.07
6,0.72,0.28,0.0,0.0,0.0,0.0,0.07,0.08
7,0.82,0.18,0.0,0.0,0.0,0.0,0.08,0.09
8,0.91,0.09,0.0,0.0,0.0,0.0,0.08,0.10
9,1.00,0.00,0.0,0.0,0.0,0.0,0.08,0.11


In [314]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_10_sh = np.round(frontier.T,2)

In [315]:
frontier_performance(frontier_10_sh,mu,shrinked_cov)


,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations
0,0.00,1.00,0.0,0.0,0.0,0.0,0.04,0.05
1,0.16,0.84,0.0,0.0,0.0,0.0,0.04,0.05
2,0.28,0.72,0.0,0.0,0.0,0.0,0.05,0.06
3,0.40,0.60,0.0,0.0,0.0,0.0,0.06,0.07
4,0.51,0.49,0.0,0.0,0.0,0.0,0.06,0.08
5,0.61,0.39,0.0,0.0,0.0,0.0,0.07,0.08
6,0.71,0.29,0.0,0.0,0.0,0.0,0.07,0.09
7,0.81,0.19,0.0,0.0,0.0,0.0,0.08,0.10
8,0.90,0.10,0.0,0.0,0.0,0.0,0.08,0.11
9,1.00,0.00,0.0,0.0,0.0,0.0,0.08,0.11


In [316]:
mu = expected_returns
cov = pln_returns[0:60][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5 = np.round(frontier.T,2)

In [317]:
frontier_performance(frontier_1_5,mu,cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations
0,0.03,0.67,0.00,0.07,0.14,0.10,0.04,0.03
1,0.24,0.52,0.03,0.12,0.00,0.10,0.05,0.04
2,0.32,0.38,0.04,0.16,0.00,0.09,0.06,0.04
3,0.40,0.27,0.05,0.20,0.00,0.09,0.06,0.05
4,0.46,0.16,0.06,0.23,0.00,0.09,0.07,0.06
5,0.53,0.06,0.06,0.26,0.00,0.08,0.07,0.07
6,0.59,0.00,0.07,0.29,0.00,0.06,0.08,0.07
7,0.67,0.00,0.04,0.29,0.00,0.00,0.08,0.08
8,0.87,0.00,0.00,0.13,0.00,0.00,0.08,0.09
9,1.00,0.00,0.00,0.00,0.00,0.00,0.08,0.10


In [318]:
mu = expected_returns
shrinked_cov = shrinkedCovariance(pln_returns[0:60][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_1_5_sh = np.round(frontier.T,2)

In [319]:
frontier_performance(frontier_1_5_sh,mu,shrinked_cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations
0,0.00,1.00,0.0,0.0,0.0,0.0,0.04,0.03
1,0.15,0.85,0.0,0.0,0.0,0.0,0.04,0.04
2,0.28,0.72,0.0,0.0,0.0,0.0,0.05,0.05
3,0.39,0.61,0.0,0.0,0.0,0.0,0.05,0.06
4,0.50,0.50,0.0,0.0,0.0,0.0,0.06,0.06
5,0.60,0.40,0.0,0.0,0.0,0.0,0.06,0.07
6,0.70,0.30,0.0,0.0,0.0,0.0,0.07,0.08
7,0.80,0.20,0.0,0.0,0.0,0.0,0.08,0.08
8,0.90,0.10,0.0,0.0,0.0,0.0,0.08,0.09
9,1.00,0.00,0.0,0.0,0.0,0.0,0.08,0.10


In [320]:
mu = expected_returns
cov = pln_returns[60:120][assets].cov()

portfolio.mu = mu
portfolio.cov = cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10 = np.round(frontier.T,2)

In [321]:
frontier_performance(frontier_5_10,mu,cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations
0,0.05,0.89,0.01,0.02,0.0,0.02,0.04,0.02
1,0.22,0.67,0.07,0.03,0.0,0.01,0.05,0.03
2,0.31,0.54,0.11,0.03,0.0,0.01,0.05,0.04
3,0.40,0.42,0.15,0.03,0.0,0.01,0.06,0.05
4,0.48,0.30,0.18,0.03,0.0,0.00,0.06,0.06
5,0.57,0.18,0.22,0.03,0.0,0.00,0.07,0.08
6,0.65,0.07,0.25,0.03,0.0,0.00,0.07,0.09
7,0.77,0.00,0.21,0.01,0.0,0.00,0.08,0.10
8,0.90,0.00,0.10,0.00,0.0,0.00,0.08,0.11
9,1.00,0.00,0.00,0.00,0.0,0.00,0.08,0.12


In [322]:
shrinked_cov = shrinkedCovariance(pln_returns[60:120][assets],shrin_param)

portfolio.mu = mu
portfolio.cov = shrinked_cov

frontier = portfolio.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

frontier_5_10_sh = np.round(frontier.T,2)

In [323]:
frontier_performance(frontier_5_10_sh,mu,shrinked_cov)

,ISAC.L,^TBSP,XAUPLN,MWIG40TR,CORP.L,PLOPLN3M,Returns,Std. deviations
0,0.00,1.00,0.0,0.0,0.0,0.0,0.04,0.03
1,0.15,0.85,0.0,0.0,0.0,0.0,0.04,0.04
2,0.27,0.73,0.0,0.0,0.0,0.0,0.05,0.05
3,0.38,0.62,0.0,0.0,0.0,0.0,0.05,0.06
4,0.49,0.51,0.0,0.0,0.0,0.0,0.06,0.07
5,0.59,0.41,0.0,0.0,0.0,0.0,0.06,0.08
6,0.70,0.30,0.0,0.0,0.0,0.0,0.07,0.09
7,0.80,0.20,0.0,0.0,0.0,0.0,0.08,0.10
8,0.90,0.10,0.0,0.0,0.0,0.0,0.08,0.11
9,1.00,0.00,0.0,0.0,0.0,0.0,0.08,0.12
